#                         Building CNN on CIFAR-10 Dataset

In [6]:
# Import Libraries
import torch.nn as nn
from torchvision import datasets,transforms,models
import numpy as np

In [ ]:
# DATA preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5))])
num_workers = 1


In [ ]:
# Check for Dataset Availability

train = datasets.CIFAR10('data',train=True,download=True,transform=transform)
test  = datasets.CIFAR10('data',train=False,download=True,transform=transform)

In [13]:
# Define Validation Parameters

valid_pct = 0.2 # The percentage of training data to be used for validation set.

total_obs = len(train) 
indxs = list(range(total_obs))
np.random.shuffle(indxs)
split = int(np.floor(valid_pct*total_obs))
train_indxs, valid_indxs = indxs[split:],indxs[:split]


80

In [14]:
# Create Samplers to randomly sample data in TRAIN and VALIDATION sets
from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_indxs)
valid_sampler = SubsetRandomSampler(valid_indxs)

In [ ]:
# Create Data Loaders

train_loader = torch.utils.data.DataLoader(dataset=train,
                                          batch_size=32,
                                          sampler=train_sampler
                                          shuffle=True,
                                          num_workers=num_workers)


valid_loader = torch.utils.data.DataLoader(dataset=train,
                                          batch_size=32,
                                          sampler=valid_sampler
                                          shuffle=True,
                                          num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(dataset=test,
                                          batch_size=32,
                                          shuffle=True,
                                          num_workers=num_workers)

In [ ]:
classes = ['airplane', 'automobile',
           'bird', 'cat',
           'deer','dog',
           'frog', 'horse',
           'ship', 'truck']

- [x] Download Datasets 
- [x] Create Loaders

## Define Network Architecture
---

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3)
        self.pool  = nn.MaxPool2d(kernel_size=2,stride=2)
        self.relu  = nn.ReLU()
        self.drop  = nn.Dropout(p=0.25)
        self.fc1   = nn.Linear(16*3*3,128)
        self.fc2   = nn.Linear(128,32)
        self.fc3   = nn.Linear(32,10)
        
        
    def forward(self,x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1,16*3*3)
        x = self.relu(self.fc1(self.drop(x)))
        x = self.relu(self.fc2(self.drop(x)))
        x = nn.LogSoftmax(self.fc3(x))
        
        return x
        
model = CNN()
model
    

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (drop): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=144, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [16]:
# Define Loss

criterion = nn.NLLLoss()

# Optimizer

optimizer = torch.optim.SGD(model.parameters(),lr=0.003,momentum=0.9)

In [ ]:
# Train the Network
epochs = 8

min_valid_loss = np.Inf

for epoch in range(1,epochs+1):
    
    train_loss =0
    valid_loss =0
    model.train()
    ############################## TRAIN ############################
    for img,label in train_loader:
        optimizer.zero_grad() # Clear the residual garbage gradients
        
        if GPU: # Move to GPU if CUDA is available
            img,label = img.cuda(),label.cuda()
            
        output = model(img) # Forward Pass
        loss = criterion(output,label) # Calculate the loss
        loss.backward() # Backpropagation step to populate the gradients
        optimizer.step() # Make the weight updates
        
        train_loss+=loss.item()*img.shape(0)
        
    ############################ VALIDATION ############################
    model.eval()
    
    for img,label in valid_loader:
        if GPU:
            img,label = img.cuda(),label.cuda()
            output = model(img) # Forward Pass
            loss = criterion(output,label) # Calculate the loss
            valid_loss += loss.item()*img.shape(0)
            
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss
        
            



###  Load the Model with the Lowest Validation Loss

In [ ]:
model.load_state_dict(torch.load('model_cifar.pt'))

---

In [19]:
# Test the Network
test_loss =0

correct_class = list(0. for i in range(10))
class_total = list(0. for i in range(10))

In [18]:
model.eval()

for data,target for test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    
    _,pred = torch.max(output,1)
    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
        
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
        


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]